In [ ]:
import re
import requests
from bs4 import BeautifulSoup as bs
from db import FileDB

# DB 정의
file_db = FileDB()

In [3]:
# 상수 정의

# 세부적인 카테고리를 대표 카테고리로 통일하기 위한 map
CATEGORY_MAP = {
        # 취업
        "강소기업채용" : "취업",
        "채용정보" : "취업",
        "인턴쉽" : "취업",
        "교육프로그램" : "취업",
        "고시반" : "취업",
        "기타" : "취업",

        # 장학
        "국가장학금" : "장학",
        "교외장학금" : "장학",
        "교내장학금" : "장학",
        "면학근로" : "장학",
        "학자금대출" : "장학",
        "국가근로" : "장학",
        "공모전 등" : "장학",
        "비교과장학" : "장학",

        # 창업
        "창업정보" : "창업",
        "창업공모전" : "창업",
        "창업행사" : "창업",
        "기타" : "창업",
    }
# RSS 피드의 페이지 번호를 지정하기 위한 기본 URL. {} 부분에 숫자가 들어감.
BASE_URL = 'https://www.hansung.ac.kr/bbs/hansung/143/rssList.do?page={}'
# 상대 경로를 절대 경로로 변환할 때 사용할 기본 도메인
BASE_DOMAIN = "https://www.hansung.ac.kr"

In [11]:
# 함수 정의

# 카테고리 정규화 함수
def normalize_category(category):
    return CATEGORY_MAP.get(category, category)

# 공지사항 게시글을 조회하여 내용, 사진, 첨부파일 수집하는 함수
def html_croll(url):
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')

    content, image_url, attachments = "", None, []
    
    # 내용 및 사진
    view_con_div = soup.find('div', class_='view-con')
    if view_con_div:
        content = view_con_div.get_text(strip=True) # 본문 텍스트 전체 가져오기
        # 그 안의 모든 텍스트를 태그 제거 + 붙여서 가져옴
        # 표(table)나 문단(p) 같은 게 전부 줄바꿈 없이 한 덩어리로 합쳐져 버림
        
        # 이미지 URL을 찾기 (content와 상관없이 이미지 URL을 추출)
        image_tag = view_con_div.find('img')
        if image_tag and 'src' in image_tag.attrs:
            image_url = image_tag['src']
    else:
        content = "No content found"

    # 첨부파일
    file_div = soup.find('div', class_='view-file')
    if file_div:
        for a_tag in file_div.find_all('a', href=True):
            href = a_tag['href']
            # 다운로드 링크만 걸러내기
            if "download.do" in href:
                file_url = f"https://www.hansung.ac.kr{href}"
                file_name = a_tag.get_text(strip=True)
                attachments.append(f"{file_name} | {file_url}") # 문자열로 합쳐서 저장
    
    return content, image_url, attachments

# RSS 피드를 순회하여 제목, 링크, 게시일, 카테고리 수집하는 함수
def rss_croll(db, max_pages, base_url, base_domain):
    saved_cnt = 0  # 저장된 공지사항 개수

    for page_number in range(1, max_pages+1):
        url = base_url.format(page_number)
        page = requests.get(url)
        soup = bs(page.text, 'xml')
        items = soup.find_all('item')

        if not items:
            print(f"{page_number} 페이지에 더 이상 게시물이 없습니다.")
            break # 게시물이 없으면 중단

        for item in items:
            title = item.find('title').get_text(strip=True) if item.find('title') else "No Title"
            link = item.find('link').get_text() if item.find('link') else "No Link"
            pub_date = item.find('pubDate').get_text(strip=True) if item.find('pubDate') else "No Date"
            category = item.find('category').get_text(strip=True) if item.find('category') else "No Category"

            # 카테고리 정규화
            category = normalize_category(category)

            # 절대 경로로 변경
            if link.startswith("/"):
                link = f"{base_domain}{link}"

            # 내용, 사진, 첨부파일들
            content, image_url, attachments = html_croll(link)

            # '143/' 뒤에 오는 숫자 그룹을 찾는 정규표현식
            match = re.search(r'143/(\d+)', link)
            notice_id = match.group(1)

            # DB에 저장
            db.save_notice(
                notice_id=notice_id,
                title=title,
                link=link,
                date=pub_date,
                category=category,
                content=content,
                image_url=image_url,
                attachments=attachments
            )
            saved_cnt += 1
            
    print(f"총 {saved_cnt}개의 공지사항이 성공적으로 저장되었습니다!")

In [12]:
# 실행

page = 1

rss_croll(
    db=file_db,
    max_pages=page,
    base_url=BASE_URL,
    base_domain=BASE_DOMAIN
)

총 30개의 공지사항이 성공적으로 저장되었습니다!
